In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
path = '/content/drive/MyDrive/aida/dataset/'
df = pd.read_csv(path+'기술과학분야_train_set.csv',index_col=0)
df_val = pd.read_csv(path+'기술과학분야_valid_set.csv',index_col=0)

In [3]:
df = df.drop(index=df[df['domain'] == '기후'].index)
df = df.reset_index(drop=True)
df_val = df_val.drop(index=df_val[df_val['domain'] == '기후'].index)
df_val = df_val.reset_index(drop=True)

In [4]:
train = pd.concat([df.loc[df['domain']=='기술과학'].sample(150000),df.loc[df['domain']=='세계'].sample(150000),df.loc[df['domain']=='정치'].sample(150000),df.loc[df['domain']=='경제'].sample(150000)]).sample(600000).reset_index(drop=True)
valid = df_val

train['labels'] = train['domain'].replace(['세계', '경제', '기술과학','정치'],[0, 1, 2, 3])
valid['labels'] = df_val['domain'].replace(['세계', '경제', '기술과학','정치'],[0, 1, 2, 3])

# 모델링

In [5]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 3.7 MB/s 
     |████████████████████████████████| 7.6 MB 36.6 MB/s 
     |████████████████████████████████| 163 kB 64.9 MB/s 


In [3]:
from tqdm import tqdm
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer,BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained("klue/bert-base",num_labels=4)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
# 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ModuleNotFoundError: ignored

In [7]:
class CustomDataset(Dataset):

    def __init__(self,text, label,tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.labels = label
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        text = str(self.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(self.labels[index], dtype=torch.float),
            }
    


training_set = CustomDataset(train['ko'],train['labels'], tokenizer, 128)
testing_set = CustomDataset(valid['ko'], valid['labels'],tokenizer, 128)

train_params = {'batch_size': 32,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 32,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [8]:
loss_fn = nn.CrossEntropyLoss() 

LEARNING_RATE = 5e-5
optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)


train_steps = len(training_loader.dataset) // train_params['batch_size']
val_steps = len(testing_loader.dataset) // test_params['batch_size']


In [ ]:
from tqdm import tqdm
epochs= 5
PATH = "C:/Users/withus/Desktop/sabin/Trend-analysis/Keyphrase_classification/model_output/"

for epoch in (range(epochs)):
    model.train()
    num_correct = 0
    trainning_loss = 0
    for i, data in enumerate(tqdm(training_loader)):
        
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        label = data['labels'].to(device, dtype = torch.long)

        optimizer.zero_grad()
        
        output = model(ids, mask, token_type_ids)
        print(output)
        loss = loss_fn(output[0], label)
        trainning_loss += loss
        loss.backward()
        optimizer.step()
        if (i+1) % 1000 == 0:
          print("Training Losses: {}".format(trainning_loss/i+1))
          print("Trainong Accuracy:{}".format(num_correct/i+1))
    
        pred = output[0].argmax(dim=1)
        num_correct += torch.eq(pred,label).sum().float().item()
        


    print('EPOCH ', epoch+1)
    print("Training Losses: {}".format(trainning_loss/len(training_loader.dataset)))
    print("Trainong Accuracy:{}".format(num_correct/len(training_loader.dataset)))
    
    
    
    if epoch % 5 == 0 or epoch == epochs-1:
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': trainning_loss,
                    }, PATH+f"model600000data{epoch}.pt")
  

: 

In [9]:
checkpoint = torch.load('/content/drive/MyDrive/aida/model600000data4.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [10]:
model.eval()
num_correct = 0

for i, data in enumerate(tqdm(testing_loader)):

      ids = data['input_ids'].to(device, dtype = torch.long)
      mask = data['attention_mask'].to(device, dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
      label = data['labels'].to(device, dtype = torch.long)


      optimizer.zero_grad()

      output = model(ids, mask, token_type_ids)
      loss = loss_fn(output[0], label)
      optimizer.step()

      pred = output[0].argmax(dim=1)
      num_correct += torch.eq(pred, label).sum().float().item()


print('EPOCH ', epoch+1)
print("validation Losses: {}".format(loss))
print("validation Accuracy:{}".format(num_correct/len(testing_loader.dataset)))


  0%|          | 0/4377 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 4377/4377 [18:33<00:00,  3.93it/s]

EPOCH  5
validation Losses: 0.012366868555545807
validation Accuracy:0.9405813347422834


# keyphrase classification(키워드 뽑은거 돌려 보는 부분 부분)


In [16]:
for i in range(1,5):
  file_path = "/content/drive/MyDrive/aida"
  if(i == 1):
    file_path += "/rent.json"
  if(i == 2):
    file_path += "/accident.json"
  if(i == 3):
    file_path += "/discrimination.json"
  if(i == 4):
    file_path += "/neutral.json"

  save_path = "/content/drive/MyDrive/aida"
  if(i == 1):
    save_path += "/rent_multi_content.json"
  if(i == 2):
    save_path += "/accident_multi_content.json"
  if(i == 3):
    save_path += "/discrimination_multi_content.json"
  if(i == 4):
    save_path += "/neutral_multi_content.json"


  with open(file_path, 'r') as file:
      keywords = json.load(file)

  category = ['사회', '경제', '기술과학','정치']
  keyword_params = {'batch_size': 5,
                  'shuffle': True,
                  'num_workers': 0
                  }
  key_and_category = []
  for keyword in tqdm(keywords):
    keyword_5 = list(keyword.values())[0]
    keyword_set = CustomDataset(keyword_5,[-1]*len(keyword_5), tokenizer, 128)
    keyword_loader = DataLoader(keyword_set, **keyword_params)
    for i, data in enumerate((keyword_loader)):

      ids = data['input_ids'].to(device, dtype = torch.long)
      mask = data['attention_mask'].to(device, dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
      label = data['labels'].to(device, dtype = torch.long)

      output = model(ids, mask, token_type_ids)
      
      pred = pd.Series(output[0].argmax(dim=1).cpu().numpy()).replace([0,1,2,3],category)
      [key_and_category.append([keyword_5[i],pred[i]]) for i in range(5)]

  with open(save_path, 'w', encoding='utf-8') as f:
          json.dump(key_and_category, f, ensure_ascii=False, indent=4)
      

  0%|          | 0/17492 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 29995/29995 [20:20<00:00, 24.58it/s]
